In [33]:
import networkx as nx
import numpy as np
from itertools import chain,combinations
import itertools
import pandas as pd
import sys
from tqdm.notebook import tqdm, trange
import time

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [34]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [35]:
def find_max_weight_edge(sim_graph):
    # Search for
    max_weight = -1
    tu = 0
    tv = 0
    vertex_count = sim_graph.number_of_nodes()
    for u in range(0,vertex_count):
        linked_nodes = sim_graph.adj[u]
        for v in list(linked_nodes):
            w = sim_graph.edges[u,v]["weight"]
            print("u="+str(u) + "," + str(v) + ",weight=" + str(w))
            if (w > max_weight):
                tu = u
                tv = v
                max_weight = w
    return tu,tv, max_weight

In [36]:
def delete_edges_from(graph, vertex):
    e = graph.adj[vertex]
    for linked in list(e):
        graph.remove_edge(vertex,linked)

In [37]:
def sort_by_metric_func(graph):
    return list(graph.nodes())

In [38]:
def get_metric_ranking(rankings, num):
    return rankings.index(num)

In [39]:
# Eqn 2 in diversification paper
def calc_compdist_from(graph, i, j, lam):
    dimension_dist = 0
    i_props = graph.nodes[i]
    j_props = graph.nodes[j]
    
    rankings = sort_by_metric_func(graph)
    ranking_i = get_metric_ranking(rankings, i)
    ranking_j = get_metric_ranking(rankings, j)
    print("ranking_i" + str(ranking_i))
    print("ranking_j" + str(ranking_j))
    
    # Get the properties
    for k,v in i_props.items():
        # Dimension values only
        if "d" in k:
            dist = (v - j_props[k])
            dimension_dist += (dist ** 2.0)
            
    comp_value = ranking_i + ranking_j + 2 * lam * np.sqrt(dimension_dist)
    print("dimension_dist = " + str(dimension_dist))
    print("comp_value = " + str(comp_value))
    return comp_value
    

In [40]:
# Need to add arbitrary vertex
def max_sum_divergence_approx(sim_graph, phy_count):
    Phy = nx.Graph() 
    lim = int(np.floor(phy_count/2))
    for i in range(0,lim):
        mu, mv, weight = find_max_weight_edge(sim_graph)
        if (mu != mv):
            print("Max weight: " + str(mu) + "->" + str(mv) + ",weight=" + str(weight))
            Phy.add_node(mu)
            Phy.add_node(mv)
            Phy.add_edge(mu,mv)
            Phy.edges[mu,mv]["weight"] = sim_graph.edges[mu,mv]["weight"]
            print("Number of edges in Phy=" + str(Phy.number_of_edges()))
            # remove other connected edges from mu and mv in the s\im_graph
            delete_edges_from(sim_graph, mu)
            delete_edges_from(sim_graph, mv)
            # check further
        
    # If phy_count is odd, add a random vertex
    # for now, only use an even phy_count
    return Phy

In [41]:
def gen_node_props(metrics_count, dim_count, metric_max, dim_max):
    nodeprops = {}
    metrics = []
    
    for di in range(0, dim_count):
        rand_dim = np.random.uniform(low=0, high=dim_max)
        nodeprops["d" + str(di)] = rand_dim
        
    # Generate the metrics
    for mi in range(0,metrics_count):
        rand_metric = np.random.uniform(low=0, high=metric_max)
        metrics.append(rand_metric)
        nodeprops["m" + str(mi)] = rand_metric
    return nodeprops

In [42]:
def create_random_sim_graph(node_count, metric_count, metric_max, dim_count, dim_max):
    sim = nx.Graph();
    for i in range(0,node_count):
        sim.add_nodes_from([(i, gen_node_props(metric_count, dim_count, metric_max, dim_max))])
        
    for i in range(0, node_count):
        for j in range(0, node_count):
            if (i != j):
                sim.add_edge(i,j)

    print("Edges: = " + str(sim.number_of_edges()))
    return sim

In [43]:
def read_node_props_from_dataframe(data, i, chosen_dimensions):
    # map the selected columns in chosen_dimensions into 'd0'
    dim_num = 0
    node_props = {}
    for d in chosen_dimensions:
        dim_id = "d" + str(dim_num)
        node_props[dim_id] = data.loc[i, d]
        dim_num += 1
    node_props['m0'] = data.loc[i, "METRIC_Q"]
    node_props['NAME'] = data.loc[i, "NAME"]
    
    # Set flag if it is on the pareto front
    node_props["TESTTAG"] = data.loc[i, "TESTTAG"]        
    return node_props

In [44]:
def create_sim_graph_from_data(filename_csv, chosen_dimensions):
    sim = nx.Graph();
    data = pd.read_csv(filename_csv)
    node_count = len(data)
    for i in range(0,node_count):
        sim.add_nodes_from([(i, read_node_props_from_dataframe(data, i, chosen_dimensions))])
        
    for i in range(0, node_count):
        for j in range(0, node_count):
            if (i != j):
                sim.add_edge(i,j)
    return sim

In [45]:
def extend_dim_boundaries_if_needed(boundaries, dim_id, value):
    if dim_id in boundaries:
        interval = boundaries[dim_id]
        if not (value in interval):
            if value < interval.left:
                boundaries[dim_id] = pd.Interval(value, interval.right)
            if value > interval.right:
                boundaries[dim_id] = pd.Interval(interval.left, value)
    else:
        boundaries[dim_id] = pd.Interval(value,value, 'both')

In [46]:
# Get the maximum and minimum of particular dimensions
def find_dimensional_boundaries(simgraph):
    boundaries = {}
    node_count = len(simgraph.nodes)
    for i in range(0, node_count):
        for k,v in simgraph.nodes[i].items():
            if "d" in k:
                extend_dim_boundaries_if_needed(boundaries, k, v)
    return boundaries

In [47]:
def normalise_value(bound_interval,v):
    return (v - bound_interval.left) / bound_interval.length

In [48]:
def normalise_dimensions(simgraph):
    boundaries = find_dimensional_boundaries(simgraph)
    simgraph_out = nx.Graph()
    node_count = len(simgraph.nodes())
    for i in range(0, node_count):
        new_props = {}
        for k,v in simgraph.nodes[i].items():
            if "d" in k:
                bound_interval = boundaries[k]
                new_props[k] = normalise_value(bound_interval, v)
            else:
                new_props[k] = v
                
        simgraph_out.add_nodes_from([(i, new_props)])
                
    for i in range(0, node_count):
        for j in range(0, node_count):
            if (i != j):
                simgraph_out.add_edge(i,j)
                
    return simgraph_out

In [49]:
# TODO: simple ranking function is the sum of the metrics
# TODO: metrics would need to be scaled somehow
def ranking_func(graph,node_id):
    props = graph.nodes[node_id]
    total = 0
    for k,v in props.items():
        if "m" in k:
            total += v
    return total

In [50]:
# dist_sum theoretical max = k * (k-1) * sqrt(param_space_num_dimesions)
# assuming all dimensions are properly normalised from zero to one!

# weight_sum theortical max = k * max_rank

In [51]:
def calc_subset_value_function(graph, subset_inds, lam):
    k = len(subset_inds)
    weight_sum = 0
    for ni in subset_inds:
    # ranking_func is w in the equation
        weight_sum += ranking_func(graph, ni)
    
    # Are we double counting the distance here?
    # dist_sum has to be normalised over the number of dimensions?
    # the 2 is to compensate for counting them only once?
    # not needed here?
    dist_sum = 0
    for ni in subset_inds:
        for nj in subset_inds:
            dist_sum += dist_func(graph, ni,nj)
            
    total = (k-1)*weight_sum + 2*lam*dist_sum
    return total

In [52]:
def dist_func(graph, ni, nj):
    i_props = graph.nodes[ni]
    j_props = graph.nodes[nj]
    dimension_dist = 0.0
    for k,v in i_props.items():
    # Dimension values only
        if "d" in k:
            dist = (v - j_props[k])
            dimension_dist += (dist ** 2.0)
    return np.sqrt(dimension_dist)

In [72]:
def subset_value(graph, subset, lam):
    value = calc_subset_value_function(graph, subset, lam)
    return value

In [54]:
def subset_cost(subset):
    return len(subset)

In [55]:
def meets_cost_constraint(subset):
    return (subset_cost(subset) <= FIXED_COST_BUDGET)

In [71]:
def subsets_viable_sorted_by_cost(simgraph, lam):
    all_subsets = list(powerset(simgraph.nodes()))
    viable_subsets = filter(meets_cost_constraint, all_subsets)
    subset_info = map(lambda subset: (subset, subset_value(simgraph, subset, lam)), viable_subsets)
    return sorted(subset_info, key=lambda info: info[1], reverse=True)

In [78]:
def best_subset_via_powerset(simgraph, lam):
    all_subsets = list(powerset(simgraph.nodes()))
    max_val = 0
    best_subset = None
    for p in tqdm(all_subsets):
        if meets_cost_constraint(p):
            v = subset_value(simgraph, p, lam)
            if v > max_val:
                max_val = v
                best_subset = p
    return (best_subset, max_val)

In [58]:
def filter_pareto_only(simgraph):
    simgraph_out = nx.Graph()
    node_count = len(simgraph.nodes())
    i2 = 0;
    for i in range(0, node_count):
        new_props = {}
        props = simgraph.nodes[i]
        if (props['TESTTAG'] == 1):
            for k,v in simgraph.nodes[i].items():
                new_props[k] = v
                new_props['orig_id'] = i
                
            simgraph_out.add_nodes_from([(i2, new_props)])
            i2+=1
                
    #for i in range(0, node_count):
    #    for j in range(0, node_count):
    #        if (i != j):
    #            simgraph_out.add_edge(i,j)
                
    return simgraph_out    

In [59]:
# Plot the surface and the graph between them
def plot_graph_with_edges_from_subset(g, edge_set):
    MAX_MARKER_SIZE = 10
    
    x = np.array(list(map(lambda n: g.nodes[n]['d0'], g)))
    y = np.array(list(map(lambda n: g.nodes[n]['d1'], g)))
    z = np.array(list(map(lambda n: g.nodes[n]['d2'], g)))
    # 3d spring layout
    pos = np.vstack((x, y, z)).T
    # Extract node and edge positions from the layout
    node_xyz = np.array([pos[v] for v in sorted(g)])
    
    node_ranks = np.array(list(map(lambda n: (MAX_MARKER_SIZE * ranking_func(g, n))**2.0, g)))
    
    edge_id_set = itertools.combinations(edge_selection, r=2)
    edge_xyz = np.array([(pos[t[0]], pos[t[1]]) for t in edge_id_set])

    # Create the 3D figure
    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")

    # Plot the nodes - alpha is scaled by "depth" automatically
    ax.scatter(*node_xyz.T, s=node_ranks, ec="w")

    # Plot the edges
    for vizedge in edge_xyz:
        ax.plot(*vizedge.T, color="tab:gray")
    
    #_format_axes(ax)
    fig.tight_layout()
    plt.show()


In [60]:
import sys
sys.maxsize

9223372036854775807

In [61]:
dd = pd.read_csv("/tmp/phytestout.csv")
(dd.loc[dd['TESTTAG'] == 1])

,NAME,TESTTAG,T1_TIME_MIDPOINT_MEAN,T2_TIME_LENGTH_MEAN,T3_TIME_MIDPOINT_VAR,P1_PARAMETER_MEAN,P2_PARAMETER_VARIANCE,O0_TOTAL_COUNT,O1_FUZZRANGE_COUNT,O2_DELAY_COUNT,O3_DELETION_COUNT,METRIC_Q
17,Test_018_10_12_2022_07_55_24,1,143.245182,3.175129,0.000000,0.031524,0.000000,1.0,1.0,0.0,0.0,53.0
252,Test_253_10_12_2022_19_22_31,1,56.135143,37.539320,0.000000,0.372524,0.000000,1.0,1.0,0.0,0.0,35.0
301,Test_302_10_12_2022_22_18_35,1,99.265669,73.968731,22.678741,0.334080,0.030044,2.0,2.0,0.0,0.0,3.0
355,Test_356_11_12_2022_00_16_58,1,89.536027,87.109398,12.949100,0.440515,0.004475,2.0,2.0,0.0,0.0,1.0
431,Test_432_11_12_2022_04_15_48,1,85.671286,51.300927,0.000000,0.418349,0.000000,1.0,1.0,0.0,0.0,4.0
712,Test_713_11_12_2022_18_36_20,1,81.202362,38.216523,0.000000,0.461795,0.000000,1.0,1.0,0.0,0.0,16.0
759,Test_760_11_12_2022_20_42_31,1,147.516360,0.007023,0.000000,0.231311,0.000000,1.0,1.0,0.0,0.0,591.0
763,Test_764_11_12_2022_21_45_54,1,147.516360,0.007023,0.000000,0.231311,0.000000,1.0,1.0,0.0,0.0,587.0
788,Test_789_11_12_2022_22_49_30,1,147.516360,0.007023,0.000000,0.145656,0.000000,1.0,1.0,0.0,0.0,573.0
800,Test_801_11_12_2022_23_53_18,1,147.516360,0.007023,0.000000,0.145656,0.000000,1.0,1.0,0.0,0.0,569.0


In [80]:
FIXED_COST_BUDGET = 6
lam = 1.0

sim = create_sim_graph_from_data("/tmp/phytestout.csv", ['T1_TIME_MIDPOINT_MEAN', 'T2_TIME_LENGTH_MEAN', 'T3_TIME_MIDPOINT_VAR'])
sim_normalised = normalise_dimensions(sim)
sim_pareto = filter_pareto_only(sim_normalised)
sim_pareto.nodes
phy = best_subset_via_powerset(sim_pareto, lam)
phy

  0%|          | 0/1048576 [00:00<?, ?it/s]

((0, 1, 6, 17, 18, 19), 39717.00943653178)

In [ ]:
use_approx_algorithm = False

plt.close()

# TODO: how to normalise for correct lambda
lam = 10.0
FIXED_COST_BUDGET = 6

if use_approx_algorithm:
    phy_best = max_sum_divergence_approx(sim, lam)    
else:
    phy = best_subset_via_powerset(sim, lam)
    
selection_to_plot = phy_best
edge_selection = selection_to_plot[0]
plot_graph_with_edges_from_subset(sim, edge_selection)